# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Liu  ->  J. Liu  |  ['J. Liu']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Arxiv has 73 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.02055


extracting tarball to tmp_2412.02055...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.02069


/tmp/ipykernel_2754/2822249172.py:52: LatexWarning: 2412.02069 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

extracting tarball to tmp_2412.02667...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.02688


extracting tarball to tmp_2412.02688...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.02688/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/tablez6' from 'tmp_2412.02688/tables/tablez6.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02082-b31b1b.svg)](https://arxiv.org/abs/2412.02082) | **An upper limit on the frequency of short-period black hole companions to Sun-like stars**  |
|| M. J. Green, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *14 pages, 12 figures. Submitted to A&A. Comments welcome. Long forms of Tables 3 and 4 are included as FITS files in the Arxiv zip folder*|
|**Abstract**|            Stellar-mass black holes descend from high-mass stars, most of which had stellar binary companions. However, the number of those binary systems that survive the binary evolution and black hole formation is uncertain by multiple orders of magnitude. The survival rate is particularly uncertain for massive stars with low-mass companions, which are thought to be the progenitors of most black hole X-ray binaries. We present a search for close black hole companions (separations less than 20 solar radii) to AFGK-type stars in TESS, i.e. the non-accreting counterparts to and progenitors of low-mass X-ray binaries. Such black holes can be detected by the tidally induced ellipsoidal deformation of the visible star, and the ensuing photometric light-curve variations. From an initial sample of 4.7 million TESS stars, we have selected 457 candidates for such variations. However, spectroscopic followup of 251 of them shows that none are consistent with a close black hole companion. On the basis of this non-detection, we determine (2 $\sigma$ confidence) that fewer than one in $10^5$ Solar-type stars in the Solar neighbourhood host a short-period black hole companion. This upper limit is in tension with a number of ``optimistic'' population models in the literature that predict short-period black hole companions around one in $10^{4-5}$ stars. Our limits are still consistent with other models that predict only a few in $10^{7-8}$.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02557-b31b1b.svg)](https://arxiv.org/abs/2412.02557) | **Dust emission from the bulk of galaxies at the Epoch of Reionization**  |
|| L. Ciesla, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            [Abridged] Our view of dust in primordial galaxies is limited towards a few tens of z~7 galaxies, pre-selected from UV-optical observations, and are thus not necessarily representative of the bulk of the sources at these redshifts. In this work, we aim at constraining the dust properties of galaxies at 6<z<12 by making the most of the A3COSMOS database in the JADES/GOODS-South field. We stacked ALMA band 6 and 7 observations of 4464 galaxies covered by the A3COSMOS database and used the measurements as constraints to perform UV-to-FIR SED modelling. We obtain tentative signals for the brightest UV galaxies (MUV<-19mag) as well as for the most massive ones (log Mstar>9) at 6<z<7, and upper limits for fainter (MUV>-19mag), lower mass sources (log Mstar<9), and at higher redshift (z>7). Fitting these 6<z<7 galaxies with ALMA constraints results in lower star formation rates (-0.4dex) and FUV attenuation (-0.5mag) for galaxies with log Mstar>8, compared to the fit without FIR. We extend the LIR vs MUV relation down to MUV=-19mag and show a tentative breakdown of the relation at fainter UV magnitudes. The positions of the JADES z~6.5 sample on the IRX versus beta and IRX versus Mstar diagrams are consistent with those of the ALPINE (z~5.5) and REBELS (z~6.5) samples, suggesting that the dust composition and content of our mass-selected sample are similar to these UV-selected galaxies. Extending our analysis of the infrared properties to z>7 galaxies, we find a non-evolution of beta with redshift in the MUV range probed by our sample (-17.24+/-0.62) and highlight the fact that samples from the literature are not representative of the bulk of galaxy populations at z>6. We confirm a linear relation between AV and 1/sSFR with a flatter slope than previously reported due to the use of ALMA constraints. Our results suggest that rapid and significant dust production has already happened by z~7.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02055-b31b1b.svg)](https://arxiv.org/abs/2412.02055) | **The hidden magnetic structures of a solar intermediate filament revealed by the injected flare material**  |
|| X. Yan, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *23 pages, 13 figures, accepted for publication in ApJ*|
|**Abstract**|            Solar filaments are spectacular objects in the solar atmosphere, consisting of accumulations of cool, dense, and partially ionized plasma suspended in the hot solar corona against gravity. The magnetic structures that support the filament material remain elusive, partly due to the lack of high resolution magnetic field measurements in the chromosphere and corona. In this study, we reconstruct the magnetic structures of a solar intermediate filament using EUV observations and two different methods, to follow the injection of hot material from a B-class solar flare. Our analysis reveals the fine-scale magnetic structures of the filament, including a compact set of mutually wrapped magnetic fields encasing the cool filament material, two groups of helical magnetic structures intertwining with the main filament, and a series of arched magnetic loops positioned along the filament. Additionally, we also find that the northern footpoints of the helical structures are rooted in the same location, while their southern footpoints are rooted in different areas. The results obtained in this study offer new insights into the formation and eruption mechanisms of solar filaments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02625-b31b1b.svg)](https://arxiv.org/abs/2412.02625) | **Identifying Hierarchically Triple Star Systems with Gaia DR3 and LAMOST**  |
|| T. He, et al. -- incl., <mark>J. Li</mark>, <mark>J. Li</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| **|
|**Abstract**|            Triple star systems are critical for understanding stellar dynamics and compact objects in astrophysics, yet confirmed hierarchical triples identified via spectroscopy remain limited. In this study, we identified 23 triple systems by cross-matching the Gaia DR3 non-single star catalog with LAMOST DR10 spectroscopic data; 18 of them are new discoveries. For two well-observed triples, we performed radial velocity curve fitting and light curve analysis to determine their orbital parameters, with inner and outer periods of 1.26 days and 656 days for one triple, and 3.42 days and 422 days for the other. We compared the results with other studies. We also analyzed the radial velocities (RVs) of these 23 triples, revealing a range of $V$ from approximately 40~km~s$^{-1}$ to 210~km~s$^{-1}$. Due to spectral resolution and detection limitations, velocity differences below 45~km~s$^{-1}$ in binaries and below 90~km~s$^{-1}$ in the inner binaries of triple systems are challenging to detect. Consequently, our detection range for inner orbital periods is restricted to 0.2--20 days, with the highest efficiency for periods under 10 days. These findings underscore the advantage of spectroscopic observations for identifying triple systems with short inner orbital periods.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02667-b31b1b.svg)](https://arxiv.org/abs/2412.02667) | **An unbiased statistical study of the metallicity of core-collapse supernovae based on VLT/MUSE integral-field-unit spectroscopy**  |
|| Q. Xi, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *12 pages, 7 figures*|
|**Abstract**|            Metallicity plays a crucial role in the evolution of massive stars and their final core-collapse supernova (CCSN) explosions. Integral-field-unit (IFU) spectroscopy can provide a spatially resolved view of SN host galaxies and serve as a powerful tool to study SN metallicities. Early transient surveys targeted bright galaxies with high star formation and SN rates; as a result, the discovered SNe are significantly biased toward high metallicities. More recently, the untargeted, wide-field transient surveys, such as ASAS-SN and ZTF, have discovered a large number of SNe without such a bias. In this work, we construct a large and unbiased sample of SNe discovered by (quasi-) untargted searches, consisting of 209 SNe of Types II (with unknown subtypes), IIP, IIn, IIb, Ib and Ic at z $\leq$ 0.02 with VLT/MUSE observations. This is currently the largest CCSN sample with IFU observations. With the strong-line method, we reveal the spatially-resolved metallicity maps of the SN host galaxies and acquire accurate metallicity measurements for the SN sites. Our results show that the SN metallicities range from 12 + log(O/H) = 8.1 to 8.7 dex, and the metallicity distributions for different SN types are very close to each other, with mean and median values of 8.4 - 8.5 dex. We carefully analysed the stochastic sampling effect, showing that our large sample size narrows the 1$\sigma$ uncertainty down to only 0.05 dex. The apparent metallicity differences among SN types are all within 3$\sigma$ uncertainties and the metallicity distributions for different SN types are all consistent with being randomly drawn from the same reference distribution. This suggests that metallicity plays a minor role in the origin of different CCSN types and some other metallicity-insensitive processes, such as binary interaction, dominate the distinction of CCSN types.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02688-b31b1b.svg)](https://arxiv.org/abs/2412.02688) | **Molecular gas and dust properties in $z>7$ quasar hosts**  |
|| F. Salvestrini, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *16 pages, 4 figures, 3 Appendix. Submitted to A&A*|
|**Abstract**|            Observational campaigns hunting the elusive reservoirs of cold gas in the host galaxies of quasars at the Epoch of Reionization (EoR) are crucial to study the formation and evolution of the first massive systems at early epochs. We present new Northern Extended Millimetre Array (NOEMA) observations tracing CO(6--5), CO(7--6) emission lines, and the underlying continuum in five of the eight quasars at redshift $z>7$ known to date, thus completing the survey of the cold molecular gas reservoir in the host galaxies of the first quasars. Combining NOEMA observations with archival Atacama Large Millimeter Array (ALMA) data available, we model the far-infrared spectral energy distribution with a modified blackbody to measure dust properties and star formation rates. We use CO and [CII] lines to derive molecular gas masses, which we compare with results from semi-analytical models and observations of galaxies at different epochs. No statistically significant detection of CO emission lines was reported for the five quasars in this sample, resulting in a relatively low amount of cold molecular gas in the host when compared with galaxies at later epochs. Nonetheless, gas-to-dust ratios are consistent with the local value, suggesting that the scaling relation between dust and cold gas holds up to $z>7$. Quasars at the EoR show star formation efficiencies which are among the highest observed so far, but comparable with that observed in luminous quasar at Cosmic Noon and that predicted for the brightest ($L_{bol}>3\times10^{46}$ erg s$^{-1}$) quasar objects drawn from the semi-analytical model GAEA. Quasar host galaxies at the EoR are undergoing an intense phase of star formation, which suggests a strong coupling between the luminous phase of the quasar and the rapid growth of the host.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02456-b31b1b.svg)](https://arxiv.org/abs/2412.02456) | **Temporal-spatial distribution of YSOs nearby Taurus region**  |
|| <mark>J. Liu</mark>, et al. |
|*Appeared on*| *2024-12-04*|
|*Comments*| *10 pages, Accepted for publication in SCPMA*|
|**Abstract**|            The Taurus region is one of the most extensively studied star-forming regions. Surveys indicate that the young stars in this region are comprised of Young Stellar Objects (YSOs) that cluster in groups associated with the molecular cloud (Grouped Young Stellar Objects, GYSOs), and some older ones that are sparsely distributed throughout the region (Distributed Young Stellar Objects, DYSOs). To bridge the age gap between the GYSOs ($\le$5 Myr) and the DYSOs (10-20 Myr), we conducted a survey to search for new YSOs in this direction. Based on infrared color excesses and Li I absorption lines, we identified 145 new YSOs. Combining these with the previously identified GYSOs and DYSOs, we constructed a sample of 519 YSOs that encompasses the entire region. Subsequently, we calculated the ages of the sample based on their proximity to the local bubble. The age versus Distance to the Local Bubble ($D_{\rm LB}$) relationship for the DYSOs shows a clear trend: the farther they are from the Local Bubble, the younger they are, which is consistent with the supernovae-driven formation scenario of the Local Bubble. The GYSOs also exhibit a mild age versus $D_{\rm LB}$ trend. However, they are significantly younger and are mostly confined to distances of 120 to 220 pc. Considering their distribution in the age versus $D_{\rm LB}$ space is well separated from the older DYSOs, they may also be products of the Local Bubble but formed in more recent and localized events.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xb7 in position 5544: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02069-b31b1b.svg)](https://arxiv.org/abs/2412.02069) | **Three Warm Jupiters orbiting TOI-6628, TOI-3837, TOI-5027 and one sub-Saturn orbiting TOI-2328**  |
|| M. T. Pinto, et al. -- incl., <mark>L. Acuña</mark>, <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-12-04*|
|*Comments*| *23 pages, 12 figures*|
|**Abstract**|            We report the discovery and characterization of three new transiting giant planets orbiting TOI-6628, TOI-3837 and TOI-5027, and one new warm sub-Saturn orbiting TOI-2328, whose transits events were detected in the lightcurves of the Transiting Exoplanet Survey Satellite \textbf{(TESS)} space mission. By combining TESS lightcurves with ground-based photometric and spectroscopic follow-up observations we confirm the planetary nature of the observed transits and radial velocity variations. TOI-6628~$b$ has a mass of 0.75$\pm$0.06~$M_\mathrm{J}$, a radius of 0.98$\pm$0.05~$R_J$ and is orbiting a metal-rich star with a period of 18.18424$\pm{0.00001}$ days and an eccentricity of 0.667$\pm0.016$, making it one of the most eccentric orbits of all known warm giants. TOI-3837~$b$ has a mass of 0.59$\pm$0.06~$M_\mathrm{J}$, a radius of 0.96$\pm$0.05~$R_J$ and orbits its host star every 11.88865$\pm$0.00003~days, with a moderate eccentricity of 0.198$^{+0.046}_{-0.058}$. With a mass of 2.01$\pm$0.13~$M_\mathrm{J}$ and a radius of 0.99$^{+0.07}_{-0.12}$ $R_J$, TOI-5027~$b$ orbits its host star in an eccentric orbit with $e$~=~0.395$^{+0.032}_{-0.029}$ every 10.24368$\pm{0.00001}$~days. TOI-2328~$b$ is a Saturn-like planet with a mass of 0.16$\pm$0.02~$M_\mathrm{J}$ and a radius of 0.89$\pm$0.04~$R_J$, orbiting its host star in a nearly circular orbit with $e$~=~0.057$^{+0.046}_{-0.029}$ at an orbital period of 17.10197$\pm{0.00001}$ days. All four planets have orbital periods above 10 days, and our planet interior structure models are consistsent a rocky-icy core with a H/He envelope, providing evidence supporting the core accretion model of planet formation for this kind of planets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.02082/./figures/upperlim-2d/figure.png', 'tmp_2412.02082/./figures/gaia-confirmation/figure.png', 'tmp_2412.02082/./figures/selection-functions/figure.png']
copying  tmp_2412.02082/./figures/upperlim-2d/figure.png to _build/html/
copying  tmp_2412.02082/./figures/gaia-confirmation/figure.png to _build/html/
copying  tmp_2412.02082/./figures/selection-functions/figure.png to _build/html/
exported in  _build/html/2412.02082.md
    + _build/html/tmp_2412.02082/./figures/upperlim-2d/figure.png
    + _build/html/tmp_2412.02082/./figures/gaia-confirmation/figure.png
    + _build/html/tmp_2412.02082/./figures/selection-functions/figure.png
found figures ['tmp_2412.02557/./lir_muv.png', 'tmp_2412.02557/./LIR_lowT_MUV_full.png', 'tmp_2412.02557/./lir_redshift.png']
copying  tmp_2412.02557/./lir_muv.png to _build/html/
copying  tmp_2412.02557/./LIR_lowT_MUV_full.png to _build/html/
copying  tmp_2412.02557/./lir_redshift.png to _build/html/
exported in  _build/html/2412.0

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmnt}[1]$
$\newcommand{\review}[1]{#1}$
$\newcommand{\porb}{P_\mathrm{orb}}$
$\newcommand{\pdot}{\dot{P}_\mathrm{orb}}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\hbeta}{H\beta}$
$\newcommand{\hei}{\ion{He}{I}}$
$\newcommand{\heii}{\ion{He}{II}}$
$\newcommand{\siii}{\ion{Si}{II}}$
$\newcommand{\filus}{\textit{u_s}}$
$\newcommand{\filgs}{\textit{g_s}}$
$\newcommand{\filrs}{\textit{r_s}}$
$\newcommand{\filis}{\textit{i_s}}$
$\newcommand{\filzs}{\textit{z_s}}$
$\newcommand{\filu}{\textit{u'}}$
$\newcommand{\filg}{\textit{g'}}$
$\newcommand{\filr}{\textit{r'}}$
$\newcommand{\fili}{\textit{i'}}$
$\newcommand{\filz}{\textit{z'}}$
$\newcommand{\filkg}{\textit{KG5}}$
$\newcommand{\inp}{{\tt input}}$
$\newcommand{\beer}{{\tt beer}}$
$\newcommand{\qmin}{{\tt qmin}}$
$\newcommand{\mmmr}{{\tt mmmr}}$
$\newcommand{\aell}{A_{\rm ell}}$
$\newcommand{\qm}{q_{\rm min}}$
$\newcommand{\fbh}{f_{\rm BH}}$
$\newcommand{\sbar}{\Bar{S}'}$
$\newcommand{\mpia}{1}$
$\newcommand{\tlv}{2}$
$\newcommand{\barilan}{3}$
$\newcommand{\edin}{4}$
$\newcommand{\caltech}{5}$
$\newcommand{\ing}{6}$
$\newcommand{\sheff}{7}$
$\newcommand{\warw}{8}$</div>



<div id="title">

# An upper limit on the frequency of short-period black hole companions to Sun-like stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.02082-b31b1b.svg)](https://arxiv.org/abs/2412.02082)<mark>Appeared on: 2024-12-04</mark> -  _14 pages, 12 figures. Submitted to A&A. Comments welcome. Long forms of Tables 3 and 4 are included as FITS files in the Arxiv zip folder_

</div>
<div id="authors">

M. J. Green, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** Stellar-mass black holes descend from high-mass stars, most of which had stellar binary companions.However, the number of those binary systems that survive the binary evolution and black hole formation is uncertain by multiple orders of magnitude.The survival rate is particularly uncertain for massive stars with low-mass companions, which are thought to be the progenitors of most black hole X-ray binaries.We present a search for close black hole companions (separation $\lesssim 20 R_\odot$ ) to AFGK-type stars in _TESS_ , i.e. the non-accreting counterparts to and progenitors of low-mass X-ray binaries. Such black holes can be detected by the tidally induced ellipsoidal deformation of the visible star, and the ensuing photometric light-curve variations. From an initial sample of $4.7\times10^6$ TESS stars, we have selected 457 candidates for such variations. However, spectroscopic followup of 251 of them shows that none are consistent with a close black hole companion.On the basis of this non-detection, we determine ( $2\sigma$ confidence) that fewer than one in $10^5$ Solar-type stars in the Solar neighbourhood host a short-period black hole companion.This upper limit is in tension with a number of "optimistic" population models in the literature that predict short-period black hole companions around one in $\sim 10^{4-5}$ stars. Our limits are still consistent with other models that predict only a few in $\sim 10^{7-8}$ .

</div>

<div id="div_fig1">

<img src="tmp_2412.02082/./figures/upperlim-2d/figure.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Two-dimensional upper limits on the frequency of black hole companions to Solar-type stars as a function of orbital period and black hole mass, $f_{\rm BH} (M_2, P_{\rm orb})$.
The existence of orbital periods close to 1 day is more tightly constrained than the existence of periods close to 3 days.
Dark companions with masses $M_2 < 3 M_\odot$ are less tightly constrained than more massive companions, but above this limit the dependence on companion mass is weak.
$\cmnt${Is it true that high-mass bhs are harder to find at fixed period?} (*fig:upperlim-2d*)

</div>
<div id="div_fig2">

<img src="tmp_2412.02082/./figures/gaia-confirmation/figure.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
The $K$-amplitudes derived via two methods from _Gaia_ data, against those measured from orbital solutions, for all targets with both measurements.
There is generally reasonable agreement, with some outlying points, as discussed in the text.
 (*fig:rv-comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2412.02082/./figures/selection-functions/figure.png" alt="Fig9" width="100%"/>

**Figure 9. -** 
Probability for simulated binary systems of various types to be accepted into the $\beer$ sample (top), $\qmin$ sample (middle), and $\mmmr$ sample (bottom), as a function of period.
Other variables ($\cos i$, $M_1$, and $M_2$) have been marginalised over.
 (*fig:selection-function-beer*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.02082"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tabularfootnotes}{$
$   \setcounter{savedfootnote}{\value{footnote}}$
$   \setcounter{footnote}{0}$
$   \newcommand{\thefootnote}{\textit{\arabic{footnote}}}$
$}$
$\newcommand{\restorefootnotes}{$
$   \setcounter{footnote}{\value{savedfootnote}}$
$   \newcommand{\thefootnote}{\arabic{footnote}}$
$}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\epsilon}{\varepsilon}$
$\newcommand{\thefootnote}{\textit{\arabic{footnote}}}$
$\newcommand{\thefootnote}{\arabic{footnote}}$
$\newcommand\cigale{{{\sc cigale}}}$
$\newcommand\microns{{ \mum}}$
$\newcommand\HI{{H{\sc i}}}$
$\newcommand\HInewcommand{{H{\sc i}-newcommand}}$</div>



<div id="title">

# Dust emission from the bulk of galaxies at the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.02557-b31b1b.svg)](https://arxiv.org/abs/2412.02557)<mark>Appeared on: 2024-12-04</mark> -  _Submitted to A&A_

</div>
<div id="authors">

L. Ciesla, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** The excess of UV bright galaxies observed at $z>10$ has been one of the major surprises from the JWST early observations.Several explanations have been proposed to understand the mild change in space density of the UV bright galaxies at these high redshifts, among them an evolution of dust attenuation properties in galaxies.However, our view of dust in primordial galaxies is limited towards a few tens of $z\sim7$ galaxies, pre-selected from UV-optical observations, and are thus not necessarily representative of the bulk of the sources at these redshifts.In this work, we aim at constraining the dust properties of galaxies at $6<z<12$ by making the most of the A $^3$ COSMOS database in the JADES/GOODS-South field.We stacked ALMA band 6 and 7 observations of 4464 JADES galaxies covered by the A $^3$ COSMOS database and used the measurements as constraints to perform UV-to-FIR SED modelling.We obtain tentative signals for the brightest UV galaxies ( $M_{\mathrm{UV}}<-19$ mag) as well as for the most massive ones ( $\log M_\star/M_\odot>9$ ) at $6<z<7$ , and upper limits for fainter ( $M_{\mathrm{UV}}>-19$ mag), lower mass sources ( $\log M_\star/M_\odot<9$ ), and at higher redshift ( $z>7$ ).Fitting these $6<z<7$ galaxies with ALMA constraints results in lower star formation rates ( $-0.4$ dex) and FUV attenuation ( $-0.5$ mag) for galaxies with $\log M_\star/M_\odot>8$ , compared to the fit without FIR.We extend the $L_{\mathrm{IR}}$ vs $M_{\mathrm{UV}}$ relation down to $M_{\mathrm{UV}}=-19$ mag and show a tentative breakdown of the relation at fainter UV magnitudes.The positions of the JADES $z\sim6.5$ sample on the infrared excess (IRX) versus $\beta$ and IRX versus $M_\star$ diagrams are consistent with those of the ALPINE ( $z\sim5.5$ ) and REBELS ( $z\sim6.5$ ) samples, suggesting that the dust composition and content of our mass-selected sample are similar to these UV-selected galaxies.Extending our analysis of the infrared properties to $z>7$ galaxies, we find a non-evolution of $\beta$ in the $M_{\mathrm{UV}}$ range probed by our sample (-17.24 $^{+0.54}_{-0.62}$ ) and highlight the fact that samples from the literature are not representative of the bulk of galaxy populations at $z>6$ .We confirm a linear relation between A $_{\rm V}$ and sSFR $^{-1}$ with a flatter slope than previously reported due to the use of ALMA constraints.Our results suggest that rapid and significant dust production has already happened by $z\sim7$ .

</div>

<div id="div_fig1">

<img src="tmp_2412.02557/./lir_muv.png" alt="Fig4" width="100%"/>

**Figure 4. -**  Infrared luminosity as a function of UV magnitude. The contours show the position of the whole sample with $L_{\mathrm{IR}}$ obtained from the fit using the ALMA constraint. Big circles with black borders are the weighted median values in four UV magnitude bins. Stacked or median values for samples from the literature are show in various shades of blue symbols \citep{Bowler24,Khusanova21}.  (*fig:lir_muv*)

</div>
<div id="div_fig2">

<img src="tmp_2412.02557/./LIR_lowT_MUV_full.png" alt="Fig7" width="100%"/>

**Figure 7. -** $L_{\mathrm{IR}}$ distributions of galaxies in the four UV magnitude bins considered in this work, assuming different dust temperatures. (*fig:lir_muv_lowT*)

</div>
<div id="div_fig3">

<img src="tmp_2412.02557/./lir_redshift.png" alt="Fig10" width="100%"/>

**Figure 10. -**  IR luminosity as a function of redshift for our sample (black contours, downward arrows indicate upper limits) and different samples from the literature \citep[shades of blue,][]{Laporte17,Laporte19,Tamura19,Inami22,Mitsuhashi24,Schouws24}. Symbols with downward arrows are upper limits. Circles with black border are our fiducial UV magnitude bins. (*fig:lirz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.02557"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

265  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
